In [2]:
# %%
# Imports and Setup
import os
import librosa
import numpy as np
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier

# %%
# Example labeled data storage (replace this with real drum-labeled audio slices)
feature_vectors = []
labels = []

# Example function to extract features from an audio chunk
def extract_features(chunk, sr):
    # Pitch (fundamental frequency)
    f0, voiced_flag, voiced_prob = librosa.pyin(chunk, sr=sr, fmin=50, fmax=2000)
    mean_f0 = np.nanmean(f0) if f0 is not None and not np.isnan(f0).all() else 0

    # Spectral centroid
    centroid = librosa.feature.spectral_centroid(y=chunk, sr=sr)
    mean_centroid = np.mean(centroid)

    # Spectral flux
    flux = librosa.onset.onset_strength(y=chunk, sr=sr)
    mean_flux = np.mean(flux)

    # MFCCs (first few coefficients) for timbre
    mfccs = librosa.feature.mfcc(y=chunk, sr=sr, n_mfcc=5)
    mean_mfccs = np.mean(mfccs, axis=1)

    # Concatenate all features
    return np.concatenate(([mean_f0, mean_centroid, mean_flux], mean_mfccs))

# %%
# Simulate training data (replace with real labeled data)
# This block ensures feature_vectors is a 2D array and labels is a 1D array
if not feature_vectors:  # If no training data exists
    feature_vectors = np.zeros((1, 8))  # Dummy feature vector with 8 features (adjust as needed)
    labels = np.array([0])  # Dummy label

feature_vectors = np.array(feature_vectors).reshape(-1, len(feature_vectors[0]))
labels = np.array(labels)

# Convert to arrays and fit an ensemble classifier
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(feature_vectors, labels)

# %%
# File paths
audio_file_path = '../songs/raw_songs/Monkey Wrench.ogg'
track_name = os.path.splitext(os.path.basename(audio_file_path))[0]
chart_path = f'../songs/chart_files/{track_name}.chart'

print(audio_file_path)
print(chart_path)
print(track_name)

# %%
# Load audio file
y, sr = librosa.load(audio_file_path)

# %%
# Initialize song metadata
CHART_RESOLUTION = 192
song_metadata = {
    "Name": f"\"{track_name}\"",
    "Artist": "\"Unknown\"",
    "Charter": "\"ACE\"",
    "Album": "\"Generated Charts\"",
    "Year": "\"2024\"",
    "Offset": 0,
    "Resolution": CHART_RESOLUTION,
    "Player2": "\"bass\"",
    "Difficulty": 0,
    "PreviewStart": 0,
    "PreviewEnd": 0,
    "Genre": "\"Rock\""
}

# Initialize data structure for chart
chart_data = {
    "Song": song_metadata,
    "SyncTrack": defaultdict(list),
    "Events": {},
    "ExpertDrums": defaultdict(list)
}

# %%
# Process the audio data
duration_seconds = librosa.get_duration(y=y, sr=sr)
print(f"Audio duration (seconds): {duration_seconds}")
print(f"Sample rate: {sr}")

# Append initial tempo and time signature to SyncTrack at tick=0
chart_data["SyncTrack"][0].append("TS 4")

# Estimate tempo
tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
ch_tempo = int(tempo * 1000)
chart_data["SyncTrack"][0].append(f"B {ch_tempo}")

# Detect onset times (in seconds)
onset_times = librosa.onset.onset_detect(y=y, sr=sr, units='time')

DRUM_MAPPING = {
    "kick": (0, 'K'),
    "snare": (1, 'R'),
    "hihat_closed": (2, 'Y'),
    "hihat_open": (3, 'B'),
    "tom_low": (2, 'Y'),
    "tom_mid": (2, 'Y'),
    "tom_high": (2, 'Y'),
    "crash": (4, 'G'),
    "ride": (4, 'G')
}

# %%
# Classify each onset using the trained classifier and extracted features
for onset_time in onset_times:
    start_sample = int(onset_time * sr)
    
    # Grab a short chunk around the onset
    chunk_duration = 0.05  # 50ms chunk duration
    end_sample = start_sample + int(chunk_duration * sr)
    
    if end_sample > len(y):
        end_sample = len(y)
    
    chunk = y[start_sample:end_sample]

    # Extract features for this chunk
    obs_features = extract_features(chunk, sr).reshape(1, -1)

    # Predict resolved drum label using the trained classifier
    predicted_label = clf.predict(obs_features)[0]

    # Map predicted label to Clone Hero notes
    note_num, flag = DRUM_MAPPING.get(predicted_label, (None, None))
    
    if note_num is not None:
        chart_tick = int(onset_time * CHART_RESOLUTION * sr / sr)
        note_str = f"N {note_num} 0{' ' + flag if flag else ''}"
        chart_data["ExpertDrums"][chart_tick].append(note_str)

# %%
# Build the .chart file text
chart_text = "[Song]\n{\n"
for key, value in chart_data["Song"].items():
    chart_text += f" {key} = {value if isinstance(value, (int, float)) else f'{value}'}\n"
chart_text += "}\n\n"

chart_text += "[SyncTrack]\n{\n"
for tick in sorted(chart_data["SyncTrack"].keys()):
    for event in chart_data["SyncTrack"][tick]:
        chart_text += f" {tick} = {event}\n"
chart_text += "}\n\n"

chart_text += "[Events]\n{\n}\n\n"

chart_text += "[ExpertDrums]\n{\n"
for tick in sorted(chart_data["ExpertDrums"].keys()):
    for note in chart_data["ExpertDrums"][tick]:
        chart_text += f" {tick} = {note}\n"
chart_text += "}"

os.makedirs(os.path.dirname(chart_path), exist_ok=True)
with open(chart_path, 'w') as f:
    f.write(chart_text)

if os.path.exists(chart_path) and os.path.getsize(chart_path) > 0:
    print(f"Chart file successfully created at: {chart_path}")
else:
    print("Error: Failed to create chart file or file is empty")


../songs/raw_songs/Monkey Wrench.ogg
../songs/chart_files/Monkey Wrench.chart
Monkey Wrench
Audio duration (seconds): 232.8
Sample rate: 22050


c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  w

Chart file successfully created at: ../songs/chart_files/Monkey Wrench.chart


c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
c:\Users\marve\.conda\envs\ace-env\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1102
  warnings.warn(
